In [1]:
import tensorflow as tf
from keras import backend as K
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
from keras.models import model_from_json
import os
import os.path as path

import tensorflow as tf

from keras import backend as K
from tensorflow.python.framework import graph_util
from collections import Counter

import numpy as np
import keras.models
from keras.models import model_from_json
from scipy.misc import imread, imresize,imshow
import matplotlib.pyplot as plt
%matplotlib inline

json_file = open('model.json','r')
loaded_model_json = json_file.read()
# json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load woeights into new model

# K.set_learning_phase(0)
model = loaded_model.load_weights("model/model.h5")
sess = K.get_session()

# output_node_name = "dense_2/Softmax" # Name of your output node

# with sess as sess:
#     init_op = tf.global_variables_initializer()
#     sess.run(init_op)
#     graph_def = sess.graph.as_graph_def()
#     output_graph_def = graph_util.convert_variables_to_constants(
#                                                                  sess,
#                                                                  sess.graph.as_graph_def(),
#                                                                  output_node_name.split(","))
#     tf.train.write_graph(output_graph_def,
#                          logdir="my_dir",
#                          name="my_model.pb",
#                          as_text=False)

# import tensorflow as tf
from google.protobuf import text_format



# gd = tf.GraphDef()

# with tf.gfile.FastGFile("my_dir/my_model.pb", "r") as f:
#     text_format.Merge(f.read(), gd)

# for node in gd.node:
#     if node.op == "Switch":
#         node.op = "Identity"
#         del node.input[1]

# tf.train.write_graph(gd, ".", "my_dir/fixed_model.pb")

MODEL_NAME = 'surh_convnet'
saver = tf.train.Saver()
input_node_names = ["conv2d_1_input"]
output_node_name = "dense_2/Softmax"


tf.train.write_graph(K.get_session().graph_def, 'out', \
                     MODEL_NAME + '_graph.pbtxt', as_text=False)

saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        True, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

input_graph_def = tf.GraphDef()
with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "r") as f:
#     input_graph_def.ParseFromString(f.read())
    text_format.Merge(f.read(), input_graph_def)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())

print("graph saved!")

C:\Users\COEGIN\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


INFO:tensorflow:Restoring parameters from out/surh_convnet.chkp
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 60: invalid start byte

In [ ]:
# import tensorflow as tf
# from google.protobuf import text_format
# from tensorflow.python.platform import gfile

# def pbtxt_to_graphdef(filename):
#   with open(filename, 'r') as f:
#     graph_def = tf.GraphDef()
#     file_content = f.read()
#     text_format.Merge(file_content, graph_def)
#     tf.import_graph_def(graph_def, name='')
#     tf.train.write_graph(graph_def, 'my_dir/', 'protobuf.pb', as_text=False)

# def graphdef_to_pbtxt(filename): 
#   with gfile.FastGFile(filename,'rb') as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())
#     tf.import_graph_def(graph_def, name='')
#     tf.train.write_graph(graph_def, 'my_dir/', 'protobuf.pbtxt', as_text=True)
#   return


# graphdef_to_pbtxt('graph.pb')